# TP1 Premiers pas avec OpenMP
## M1 informatique, Université d'Orléans 2021/2022

*L'objectif de ce TP est de découvrir [OpenMP](https://www.openmp.org/) en expérimentant les différentes directives depuis les plus bas niveaux jusqu'aux constructions les plus sophistiquées autour des boucles.*

### 0. Fiches de TP

Les énoncés des TP sont fournis sous forme de *notebook Jupyter*. Il s'agit de documents modifiables. Lorsque le document est visualisé dans un navigateur web, des menus (et éventuellement une barre d'outil si elle est activée) permettent de manipuler le document.

Le document est composé d'une suite de cellules qui peuvent contenir du texte mis en forme au [format Markdown](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd), du code Python ou encore du texte brut. Ces cellules peuvent être *déplacées*, de nouvelles cellules **insérées**, etc.

*PS. Pour ceux d'entre nous qui préférent travailler en mode texte ou dans un terminal, il est possible de travailler sur le source de la fiche de TP. On perd simplement la possibilité d'exécuter directement du code Python au fil de l'eau dans la fiche.*

## 1. Hello parallel world!

Pour cette première partie, il suffit de disposer d'un compilateur C++ avec support pour OpenMP, d'un terminal ouvert et d'un éditeur de fichiers.

Commençons par créer un fichier `Makefile` pour préciser quel compilateur utiliser et quels arguments lui passer à la compilation. Nous n'écrirons pas de règles, notre source n'ayant aucune dépendance, les règles implicites suffiront pour compiler notre programme.

```make
CXX=g++
CXXFLAGS=-std=c++11 -Wall -Wextra -pedantic -fopenmp
```

Et voici notre premier programme OpenMP `hello.cpp`

~~~c++
#include <iostream>
#include <omp.h>
using namespace std;

int main(int, char *[])
{
    cout << "Bonjour" << endl;
    cout << "Nous sommes " << omp_get_num_threads() << " threads dans cette équipe" << endl;
    cout << "Au revoir" << endl;
}
~~~

Si tout se passe bien, la commande `make hello` devrait compiler ce programme et sans surprise au lancement on obtient un comportement très séquentiel.

```
$ ./hello 
Bonjour
Nous sommes 1 threads dans cette équipe
Au revoir
```

### 1.1. pragma omp parallel

La directive `#pragma omp parallel` indique que l'instruction (ou le bloc d'instructions) qui suit doit être exécuté par une équipe de *threads* (souvenez-vous qu'openMP repose sur le paradigme de programmation parallèle [fork-join](https://en.wikipedia.org/wiki/Fork–join_model)).

La fonction de bibliothèque `omp_get_num_threads` permet de connaître le nombre total de *threads* dans l'équipe et la fonction `omp_get_thread_num` indique le numéro du *thread* courant. 

La variable d'environnement `OMP_NUM_THREADS` permet de spécifier le nombre de *threads* souhaités. *Il est aussi possible d'utiliser la fonction de la bibliothèque `omp_set_num_threads` mais l'utilisation d'une variable d'environnement permet de changer la valeur plus facilement sans recompiler le programme.*

*Les variables déclarées à l'intérieur d'un bloc parallèle sont par défaut privées : chaque thread dispose de sa propre variable. Les variables déclarées à l'extérieur d'un bloc parallèle et celles allouées sur le tas sont partagées par tous les threads de l'équipe.*

### 1.2. pragma omp critical, single, barrier

La directive `#pragma omp critical` définit une section critique, une portion de code qui ne peut être accédée par les *threads* qu'un seul à la fois.

La directive `#pragma omp barrier` définit une barrière de synchronisation, un rendez-vous que tous les *threads* doivent avoir rejoint avant d'être autorisés à poursuivre leur exécution.

La directive `#pragma omp single` définit une section de code qui sera exécutée par un unique *thread*.

### 1.3. On récapitule et on mesure le temps de calcul !

Voici un programme `syracuse.cpp` que nous allons paralléliser. Il utilise la fonction de bibliothèque `omp_get_wtime` pour mesurer le temps réel écoulé entre deux points du programme.

~~~c++
#include <iostream>
#include <omp.h>
using namespace std;

#define TOTAL 2000000

int main(int, char *[])
{
    int pass=0, cur;
    double start = omp_get_wtime();
///////////////////// MODIFIER A PARTIR D'ICI UNIQUEMENT /////////////////////
    for(int i=0; i<TOTAL; i++) {
        cur=i;
        while (cur>1)
            cur=cur%2?3*cur+1:cur/2;
        pass++;
    }
///////////////////// MODIFIER JUSQU'ICI UNIQUEMENT /////////////////////
    double end = omp_get_wtime();
    cout << pass << " out of " << TOTAL << "! (delta=" << TOTAL-pass << ")" << endl;
    cout << "ellapsed time: " << (end-start)*1000 << "ms" << endl;
}
~~~

Si votre programme effectue une section critique à chaque passage dans la boucle... il est certainement peu efficace et la parallélisation peut même aller jusqu'à dégrader le temps de calcul par rapport au séquentiel !

*PS. On pourra s'aider d'une boucle shell comme ceci :*
```
$ for np in 1 2 4 8 16; do OMP_NUM_THREADS=$np ./syracuse; done
```

## 2. Vers l'infini et au-delà

Dans cette deuxième partie, nous allons paralléliser un code existant. Téléchargez et décompressez l'archive
`astro.tar.gz` fournie avec le TP. **Lire le fichier `README.md`** — dans un premier temps, vous pouvez laisser de côté la partie `bench` et `show` qui servira à effectuer des mesures de performance.

Le programme à paralléliser prend en entrée une image au format [FITS](https://fr.wikipedia.org/wiki/Flexible_Image_Transport_System) $\operatorname{Astro}$ et produit en sortie une image $\operatorname{Resca}$ au format [PGM](https://fr.wikipedia.org/wiki/Portable_pixmap) produite par une mise à l'échelle comme ceci :

$$
\forall i,j,\quad \operatorname{Resca}(i,j) = 255\frac{\operatorname{Astro}(i,j)-m}{M-m}\qquad\mbox{où }
\begin{cases}M=\max_{i,j}\operatorname{Astro}(i,j) \\ m=\min_{i,j}\operatorname{Astro}(i,j)\end{cases}
$$

Ces calculs sont effectués dans la partie du code qui est à modifier :

~~~c++
    ///////////////////// MODIFIER A PARTIR D'ICI UNIQUEMENT /////////////////
    for (j = 0; j < astro.height(); j++)
      for (i = 0; i < astro.width(); i++) {
        amin = min(amin, astro(i, j));
        amax = max(amax, astro(i, j));
      }
    unsigned short arange = amax - amin;
    for (j = 0; j < astro.height(); j++)
      for (i = 0; i < astro.width(); i++)
        resca(i, j) = (astro(i, j) - amin) * 255 / arange;
    ///////////////////// MODIFIER JUSQU'ICI UNIQUEMENT /////////////////////
~~~


**Astuce.** Pour comparer les deux images `resca-ref.pgm` et `resca.pgm`, vous pouvez utiliser par exemple l'outil de comparaison de *Image Magick* ou *Graphics Magick* si vous en disposez :

```
$ compare /tmp/resca-ref.pgm /tmp/resca.pgm /tmp/diff.png && xdg-open /tmp/diff.png
$ magick compare /tmp/resca-ref.pgm /tmp/resca.pgm /tmp/diff.png && xdg-open /tmp/diff.png
$ gm compare /tmp/resca-ref.pgm /tmp/resca.pgm /tmp/diff.png && xdg-open /tmp/diff.png
```

Sinon vous pouvez de manière plus minimaliste et comparer les empreintes des deux fichiers pour tester leur égalité :

```
$ shasum /tmp/resca-ref.pgm /tmp/resca.pgm
```

### 2.1. pragma omp for, collapse

Les boucles sont tellement fréquentes dans le code numérique à paralléliser qu'OpenMP dispose de directives dédiées à cette tâche. Ainsi `#pragma omp for` distribue les indices de la boucle qui suit la directive entre les différents *threads* de l'équipe et nous libère des fastidieux calculs associés.

Lorsque plusieurs boucles sont imbriquées, la clause `collapse(N)` permet de paralléliser simultanément les `N` premiers niveaux de boucles. Dans notre exemple il peut être pertinent de traiter les deux boucles simultanément.

### 2.2. reduction

Notre premier bloc de boucles imbriquées calcule un minimum et un maximum. Effectuer ce calcul dans chaque *thread* avant de mettre en commun le résultat est une opération très courante. OpenMP permet d'automatiser cette phase grâce à la clause `reduction(op:var)` où `op` est l'opération qu'on souhaite réduire et `var` la variable où stocker le résultat.

Enfin, il est possible de contracter les directives `#pragma omp parallel` et `#pragma omp for` lorsqu'elles se suivent en un unique `#pragma omp parallel for`.

### 2.3. schedule

La manière dont les *threads* se partagent les indices impacte les performances du calcul. OpenMP permet de choisir une stratégie de distribution des indices à travers la clause `schedule(S)` où `S` décrit la politique à appliquer. Les stratégies les plus classiques sont :
 - `schedule(static)` : découpe les indices en autant de blocs de même taille qu'il y a de *threads* ;
 - `schedule(static,N)` : découpe les indices en blocs de taille `N` et les distribue équitablement et cycliquement entre les *threads*  ;
 - `schedule(dynamic,N)` : découpe les indices en blocs de taille `N` et les attribue successivement aux *threads* au fur et à mesure qu'ils sont disponibles ;
 - `schedule(runtime)` : applique la stratégie décrite dans la variable d'environnement `OMP_SCHEDULE`.

À l'aide des deux variables `OMP_SCHEDULE` et `OMP_NUM_THREADS`, il est possible d'étudier l'influence de la stratégie de distributions d'indices et de chercher les meilleurs paramètres pour une machine donnée. C'est ce que proposent les scripts `bench`, pour collecter des statistiques, et `show`, pour afficher ces données sous forme de courbes de performance.

Si $T_S$ est le temps de calcul séquentiel et $T_P(p)$ le temps de calcul parallèle pour effectuer une tâche avec $p$ *threads*, on définit l'accélération comme la quantité $S^*(p) = \frac{T_S}{T_P(p)}$ et l'efficacité comme $E^*(p) = \frac{S^*(p)}{p}$.


Si vous ne disposez pas des bibliothèques pour générer les courbes, vous pouvez les générer dans ce *notebook* en collant le contenu de `stats.csv` puis en exécutant les cellules ci-dessous.


In [ ]:
from io import StringIO
from ezplot import ezplot

In [ ]:
data="""
### COLLER A LA PLACE DE CETTE LIGNE LE CONTENU DE stats.csv
"""

In [ ]:
ezplot(StringIO(data),['bs'])

In [ ]:
ezplot(StringIO(data),['schedule'])

## Références

 1. [Spécification OpenMP](https://www.openmp.org/specifications/)
 1. [OpenMP Reference Guide](https://www.openmp.org/resources/refguides/)
 1. [Exemples de statistiques collectées pour la partie 2](?from=progpar/perf.ipynb&module=progpar/ezplot.py) !